In [ ]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.obj import EventLog, Trace, Event
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
from pm4py.objects.petri_net import semantics
import random

# ======================
# Import Log
# ======================
log = xes_importer.apply(
	"C:/Users/Simone/Desktop/UNIVERSITA/MAGISTRALE/BIOMEDICAL DECISION SUPPORT SYSTEM/Assignments/2/log-10-percent-noise.xes.gz"
)

# ======================
# Variants
# ======================
from pm4py.statistics.variants.log import get as variants_get
variants = variants_get.get_variants(log)
print("Number of variants: ", len(variants))

# ======================
# Train/Test Split
# ======================
test_log = log[10:]
train_log = log[0:10]


# ======================
# Utility: Create EventLog
# ======================
def create_event_log(traces_list):
	log = EventLog()
	for trace_data in traces_list:
		trace = Trace()
		for event_data in trace_data[:-1]:
			event = Event(event_data)
			trace.append(event)
		log.append(trace)
	return log


train = create_event_log(train_log)
test = create_event_log(test_log)

print("First trace, first event:", train[0][0])

# ======================
# Discover Petri Net
# ======================
net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(train)
# pm4py.view_petri_net(net, initial_marking, final_marking)

# ======================
# Perform Alignments
# ======================
aligned_traces = alignments.apply_log(train, net, initial_marking, final_marking)

# estraggo l’allineamento della prima traccia come esempio
al = [move for move in aligned_traces[0]["alignment"]]

# ======================
# Build Perfect Aligned Log
# ======================
perfect_aligned_log = []

for aligned in aligned_traces:
	aligned_trace = []
	for move_log, move_model in aligned["alignment"]:
		if move_log not in [None, ">>"] and move_model not in [None, ">>"]:
			# tengo le mosse sincronizzate
			aligned_trace.append((move_log, move_model))
		elif move_log in [None, ">>"] and move_model not in [None, ">>", "tau"]:
			# inserisco dummy for visible model moves
			aligned_trace.append((f"{move_model}_DUMMY", move_model))
		# altrimenti scarto
	perfect_aligned_log.append(aligned_trace)

print("Aligned traces (example):", aligned_traces[4])
print("Perfect aligned traces (example):", perfect_aligned_log[1])
print("Alignment of the first trace:", al[0])

print("Enumerated first alignment:")
for i, step in enumerate(aligned_traces[0]["alignment"]):
	print(i, step)

print("Total traces:", len(aligned_traces))

print("Perfect aligned log:", perfect_aligned_log)

# ======================
# Automata Learning Wrapper
# ======================
random.seed(42)


class AutomataLearningWrapper:
	def __init__(self, random_aligned, net, initial_marking, final_marking):
		self.alignment = random_aligned
		self.net = net
		self.current_marking = initial_marking
		self.current_index = 0
		self.final_marking = final_marking

	def step(self):
		# fine della sequenza
		if self.current_index >= len(self.alignment):
			return None, self.final_marking

		# prendo la transizione dal modello
		model_transition_name = self.alignment[self.current_index][1]
		transition = next(
			(t for t in self.net.transitions if t.label == model_transition_name),
			None,
		)

		if transition is None:
			print(f"Warning: transition {model_transition_name} not found in net")
			self.current_index += 1
			return None, self.current_marking

		enabled = semantics.enabled_transitions(self.net, self.current_marking)
		print("Transition to fire:", transition)
		print("Enabled transitions:", enabled)

		if transition not in enabled:
			self.current_index += 1
			return None, self.current_marking

		# eseguo transizione
		self.current_marking = semantics.execute(
			transition, self.net, self.current_marking
		)
		self.current_index += 1

		return transition, self.current_marking


# ======================
# Test Automata Learning Wrapper
# ======================
random_aligned = random.choice(perfect_aligned_log)
print("Randomly selected aligned trace for testing:", random_aligned)
print("-----")

automata_learning = AutomataLearningWrapper(
	random_aligned, net, initial_marking, final_marking
)

for i in range(0, 20):
	transition, current_marking = automata_learning.step()
	print(f"Step {i}: Transition={transition}, Marking={current_marking}")
	print("-----")